0.0 Import Packages

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as T
import os
import glob
import seaborn as sns
import fnmatch
import tensorflow as tf
import csv
import random


1.0 Testing data loading

In [2]:
loc = "Nasa Battery Data\Battery 5\Discharge cycle\Headers\B0005 cycle 4 data with headers.csv"
data = pd.read_csv(loc)
cycle1 = pd.DataFrame(data)
cycle1
#Issue is, thats actually only one example

,Voltages,Currents,Temperatures,Charge Currents,Charge Voltages,Time step,Capacity,SOH
0,4.189773,0.000021,24.697752,-0.0006,0.000,0.000,1.846327,0.923164
1,4.188881,-0.000879,24.685948,-0.0006,4.204,16.672,1.846327,0.923164
2,3.979157,-2.014654,24.738132,-1.9984,3.056,35.703,1.846327,0.923164
3,3.956924,-2.013703,24.897071,-1.9982,3.024,53.828,1.846327,0.923164
4,3.940054,-2.012797,25.084750,-1.9984,3.003,71.891,1.846327,0.923164
...,...,...,...,...,...,...,...,...
172,3.127951,-2.011634,37.886988,-1.9982,2.221,3210.703,1.846327,0.923164
173,3.083314,-2.010339,38.020076,-1.9982,2.180,3230.375,1.846327,0.923164
174,3.030431,-2.011630,38.179273,-1.9982,2.133,3250.032,1.846327,0.923164
175,2.965202,-2.012772,38.345628,-1.9982,2.075,3269.688,1.846327,0.923164


1.1 Function testing

In [3]:
#Converting pandas data frame to an array, and adding the cycle number
#wowee that was alot of work
rows = len(cycle1)
arraycycle1 = cycle1.values
cycleListold = np.asarray([2]*rows)
cycleList = np.reshape(cycleListold,(rows, 1))
addition = np.full((rows,1),cycleList)
newarraycycle1 = np.append(arraycycle1, addition, axis=1)
print(newarraycycle1)

[[ 4.1897732e+00  2.1251180e-05  2.4697752e+01 ...  1.8463272e+00
   9.2316362e-01  2.0000000e+00]
 [ 4.1888814e+00 -8.7947127e-04  2.4685948e+01 ...  1.8463272e+00
   9.2316362e-01  2.0000000e+00]
 [ 3.9791567e+00 -2.0146540e+00  2.4738132e+01 ...  1.8463272e+00
   9.2316362e-01  2.0000000e+00]
 ...
 [ 3.0304311e+00 -2.0116295e+00  3.8179273e+01 ...  1.8463272e+00
   9.2316362e-01  2.0000000e+00]
 [ 2.9652015e+00 -2.0127719e+00  3.8345628e+01 ...  1.8463272e+00
   9.2316362e-01  2.0000000e+00]
 [ 2.8782908e+00 -2.0139667e+00  3.8523283e+01 ...  1.8463272e+00
   9.2316362e-01  2.0000000e+00]]


1.2 Counting number of files

In [4]:
count = 0
for root_dir, cur_dir, files in os.walk(r'Nasa Battery Data\Battery 5\Discharge Cycle\Headers'):
    count += len(files)
print('file count:', count)


file count: 173


I'm not sure what this is

In [5]:
items = os.listdir('Nasa Battery Data\Battery 5\Discharge Cycle\Headers')

#Extract the maximum time step
filename = "Nasa Battery Data\Battery 5\Discharge cycle\Headers\B0005 Cycle 2 data with headers.csv"
data = pd.read_csv(filename)
firstcycle = pd.DataFrame(data)
fc = firstcycle["Time step"]
last_value = fc.iat[-1]
print(fc)
print(items)

#fuck yeah i did it

0         0.000
1        16.781
2        35.703
3        53.781
4        71.922
         ...   
173    3229.016
174    3248.625
175    3268.328
176    3287.969
177    3307.625
Name: Time step, Length: 178, dtype: float64
['B0005 cycle 10 data with headers.csv', 'B0005 cycle 102 data with headers.csv', 'B0005 cycle 106 data with headers.csv', 'B0005 cycle 110 data with headers.csv', 'B0005 cycle 114 data with headers.csv', 'B0005 cycle 118 data with headers.csv', 'B0005 cycle 12 data with headers.csv', 'B0005 cycle 122 data with headers.csv', 'B0005 cycle 126 data with headers.csv', 'B0005 cycle 130 data with headers.csv', 'B0005 cycle 134 data with headers.csv', 'B0005 cycle 136 data with headers.csv', 'B0005 cycle 14 data with headers.csv', 'B0005 cycle 140 data with headers.csv', 'B0005 cycle 144 data with headers.csv', 'B0005 cycle 148 data with headers.csv', 'B0005 cycle 150 data with headers.csv', 'B0005 cycle 154 data with headers.csv', 'B0005 cycle 158 data with headers.csv', 'B

In [6]:
#Okay, I need to confirm which cycle has the longest timestep


#Starting with the first cycle so that things are intilized, after that I'll be appending
filePath = "Nasa Battery Data\Battery 5\Discharge cycle\Headers\B0005 cycle 2 data with headers.csv"
data = pd.read_csv(filePath)
fullcycle = pd.DataFrame(data)

#Grabbing just the voltage data in one data frame 
#Grabbing the capacity data into another data frame
#Grabbing the last time step value to pad the other data with
cycle = fullcycle["Voltages"]
output = fullcycle["Capacity"]
cycleOneEnd = fullcycle["Time step"].iat[-1]
longestCycle = cycleOneEnd
longestCycleNum = 2


numCycles = 1 # I'm differentiating between the data set cycle number and the number of discharge cycles.
#I'm not sure if it matters but I'm doing it
count = 3
while count < 615: #find a way to automatically assign the limit later
    for fileName in os.listdir('Nasa Battery Data\Battery 5\Discharge Cycle\Headers'):

        #Increase counter until we've hit a cycle number that is in this folder
        #That means it is a discharge cycle
        cstring = str(count)
        #Create filePath variable to this file that we want to extract data from
        filePath = 'B0005 cycle ' + cstring+ ' data with headers.csv'
        

        if fnmatch.fnmatch(fileName, filePath):

            filePathFull = 'Nasa Battery Data\Battery 5\Discharge Cycle\Headers\B0005 cycle ' + cstring+ ' data with headers.csv'

            #Pull the data from the file into a data frame
            data = pd.read_csv(filePathFull)
            fullcycle = pd.DataFrame(data)

            #Now I need to pad the data until the time equals the longest time step



            #Grabbing just the voltage data in one data frame 
            #Grabbing the capacity data into another data frame
            cycle = fullcycle["Voltages"]
            output = fullcycle["Capacity"]
            time = fullcycle["Time step"]

            #Grabbing the time step and last values of the current cycle being unpacked
            timestep = time.iat[1]
            cap = output.iat[1]
            


            #Now to pad the data such that each trend continues to the same time step
            last_time = fullcycle["Time step"].iat[-1]
            last_voltage = fullcycle["Voltages"].iat[-1]

            if last_time > cycleOneEnd:
                longestCycle = last_time
                longestCycleNum = count
            





            numCycles +=1



    count += 1
    #oh fuck yeah I did it
print(longestCycleNum)
print(longestCycle)

86
3307.843


1.3 Reading in data, padding it, resampling, and uploading to new file

In [17]:
items = os.listdir('Nasa Battery Data\Battery 5\Discharge Cycle\Headers')
#it wont import in order bc github is a dink
# at least that has all the items
#now lets order it incase that matters


numCycles = 0
count = 0
while count < 615: #find a way to automatically assign the limit later
    for fileName in os.listdir('Nasa Battery Data\Battery 5\Discharge Cycle\Headers'):

        #Increase counter until we've hit a cycle number that is in this folder
        #That means it is a discharge cycle
        cstring = str(count)
        #Create filePath variable to this file that we want to extract data from
        filePath = "B0005 cycle " + cstring+ " data with headers.csv"
        

        if fnmatch.fnmatch(fileName, filePath):
            numCycles +=1
    count +=1

#Creating the model input array which is based on the number of cycles 
#Opening the file to be written to 
# open the file in the write mode
f = open('Nasa Battery Data\\Battery 5\\Discharge cycle\\Padded and Resampled\\NoisyInputData.csv', 'w', newline='')
ft = open('Nasa Battery Data\\Battery 5\\Discharge cycle\\Padded and Resampled\\NoisyInputTempData.csv', 'w', newline='')

# create the csv writer
writer = csv.writer(f)
Tempwriter = csv.writer(ft)

#Starting with the first cycle so that things are intilized, after that I'll be appending
filePath = "Nasa Battery Data\Battery 5\Discharge cycle\Headers\B0005 cycle 2 data with headers.csv"
data = pd.read_csv(filePath)
fullcycle = pd.DataFrame(data)

#Grabbing just the voltage data in one data frame 
#Grabbing the capacity data into another data frame
#Grabbing the last time step value to pad the other data with
cycle = fullcycle["Voltages"]
output = fullcycle["Capacity"]
temp = fullcycle["Temperatures"]
time = fullcycle["Time step"]
last_value = fullcycle["Time step"].iat[-1]

cap = output.iat[1]


rows = len(cycle)#Grab the number of rows in the data frame
#arraycycle = cycle.values #Convert data frame to array
#cycleListold = np.asarray([2]*rows) #Using the number of cycles, create an array containing the cycle number of this file
#cycleList = np.reshape(cycleListold,(rows, 1)) # Reshape so the program stops throwing errors
#addition = np.full((rows,1),cycleList)#do it again because it was still mad at me
#newarraycycle = np.append(arraycycle, addition, axis=1)#Add the cycle number to the data array

#allInput = newarraycycle
#listCycle = cycle.values.tolist()
#listOutput = output.values.tolist()
#listTime = time.values.tolist()

#listCycle.append(cap)
#writer.writerow(listCycle) #Writing first cycle

NoisyCycle = []
NoisyTemp = []

#allInput = [[listCycle]]
#allOutput = [[listOutput]]
#allTime = [[listTime]]


numCycles = 1 # I'm differentiating between the data set cycle number and the number of discharge cycles.
#I'm not sure if it matters but I'm doing it
count = 1
while count < 615: #find a way to automatically assign the limit later
    NoisyCycle.clear()
    NoisyTemp.clear()
    for fileName in os.listdir('Nasa Battery Data\Battery 5\Discharge Cycle\Headers'):

        #Increase counter until we've hit a cycle number that is in this folder
        #That means it is a discharge cycle
        cstring = str(count)
        #Create filePath variable to this file that we want to extract data from
        filePath = 'B0005 cycle ' + cstring+ ' data with headers.csv'
        

        if fnmatch.fnmatch(fileName, filePath):

            filePathFull = 'Nasa Battery Data\Battery 5\Discharge Cycle\Headers\B0005 cycle ' + cstring+ ' data with headers.csv'

            #Pull the data from the file into a data frame
            data = pd.read_csv(filePathFull)
            fullcycle = pd.DataFrame(data)

            #Now I need to pad the data until the time equals the longest time step



            #Grabbing just the voltage data in one data frame 
            #Grabbing the capacity data into another data frame
            cycle = fullcycle["Voltages"]
            output = fullcycle["Capacity"]
            time = fullcycle["Time step"]
            temp = fullcycle["Temperatures"]

            #They are dataframes

            #Grabbing the time step and last values of the current cycle being unpacked
            timestep = time.iat[1]
            cap = output.iat[1]


            #Converting to lists
            listCycle = cycle.values.tolist()
            listTime = time.values.tolist()
            listOutput = output.values.tolist()
            listTemp = temp.values.tolist()
        

            #Now to pad the data such that each trend continues to the same time step
            last_time = fullcycle["Time step"].iat[-1]
            last_voltage = fullcycle["Voltages"].iat[-1]
            last_temp = fullcycle["Temperatures"].iat[-1]

            while last_time < last_value:
                listCycle.append(last_voltage)
                listTime.append(last_time+timestep)
                listTemp.append(last_temp)
                last_time += timestep

            #listCycle.append(cap)

            #listTime is available for use but not printed to the data set
            #allTime.append([listTime])
            #allInput.append([listCycle]) # this should be the python variable equiv to the exported csv
            #allOutput = [[listOutput]]

            #Converting voltage lists back to data frame
            timeOrderedInput = list(zip(listTime, listCycle))
            paddedDF = pd.DataFrame(listCycle, columns = ['Voltage'])
            timeIndex = pd.TimedeltaIndex(data=listTime, unit='s')
            paddedIndexedDF = paddedDF.set_index(timeIndex)

            #Resample
            resampled = paddedIndexedDF.resample('30S').mean()
            cycle = resampled["Voltage"]

            newlistCycle = cycle.values.tolist()
            copyCycle = resampled["Voltage"].values.tolist()
            newlistCycle.append(cap)

            
            writer.writerow(newlistCycle)

            #Adding Noise

            
            for i in copyCycle:
                rand = random.uniform(-0.1, 0.1)
                newvalue = i+rand
                NoisyCycle.append(newvalue)
            NoisyCycle.append(cap)

            writer.writerow(NoisyCycle)



            #Repeating for temperature data
            timeOrderedInputTemp = list(zip(listTime, listTemp))
            paddedDFTemp = pd.DataFrame(listTemp, columns = ['Temperatures'])
            timeIndexTemp = pd.TimedeltaIndex(data=listTime, unit='s')
            paddedIndexedDFTemp = paddedDFTemp.set_index(timeIndex)

            #Resample
            resampledTemp = paddedIndexedDFTemp.resample('30S').mean()
            Temp = resampledTemp["Temperatures"]

            newlistTemp = Temp.values.tolist()
            copyTemp = resampledTemp["Temperatures"].values.tolist()
            newlistTemp.append(cap)

            
            Tempwriter.writerow(newlistTemp)


            #Adding Noise

            for i in copyTemp:
                rand = random.uniform(-0.1, 0.1)
                newvalue = i+rand
                NoisyTemp.append(newvalue)
            NoisyTemp.append(cap)

            Tempwriter.writerow(NoisyTemp)



            numCycles +=1
            #Now this individual cycle needs to be added to the larger data array
            #But the data needs to be nested correctly 
            #This won't be the version that goes to the model bc we need the resampled equal length stuff, but yeah'"Nasa Battery Data"


            #Loading all voltage data into the input array
            #And all the capacity data into the output array

            #allInput = np.append(allInput,arraycycle, axis=0) # This isn't the right shape anyway

            #I think I need to switch to tensors so I can properly nest this pad boy




    count += 1
    #oh fuck yeah I did it

f.close()
ft.close()

#technically that worked!


In [ ]:

#timeOrderedInput = list(zip(listTime, listCycle))
df = pd.DataFrame(listCycle, columns = ['Voltage'])
timeIndex = pd.TimedeltaIndex(data=listTime, unit='s')
df2=df.set_index(timeIndex)
print(df2)


resampled = paddedIndexedDF.resample('30S').mean()
cycle = resampled["Voltage"]
print(cycle)

In [16]:
newlistCycle

[4.086226575,
 3.9278400333333336,
 3.892792033333334,
 3.8671215666666665,
 3.8466937333333333,
 3.827290275,
 3.8109491,
 3.7987115,
 3.7872931,
 3.7761016,
 3.762699025,
 3.748859133333333,
 3.7364485666666667,
 3.7239535666666668,
 3.7111262333333332,
 3.696644325,
 3.6826174666666667,
 3.6711687333333334,
 3.6601752,
 3.6497048,
 3.637827575,
 3.6263267,
 3.6167789,
 3.607114733333333,
 3.5976746666666664,
 3.5870335,
 3.5766146666666665,
 3.5679391333333332,
 3.5593419666666666,
 3.5509375999999997,
 3.541518,
 3.532388766666667,
 3.5246037666666665,
 3.517126166666667,
 3.5098715666666664,
 3.501662375,
 3.4937275,
 3.4868981,
 3.4805911333333337,
 3.4742194333333334,
 3.466868375,
 3.4598883333333332,
 3.4539942666666668,
 3.4481018666666667,
 3.442428,
 3.43584325,
 3.4293479333333337,
 3.4236993333333334,
 3.4181513,
 3.411562825,
 3.405022033333333,
 3.3994517666666666,
 3.393725266666667,
 3.3877662,
 3.380803475,
 3.373571966666667,
 3.3673153,
 3.3606759333333334,
 3.3540

In [ ]:
#Which cycle has the most entries?
#How many is it?
#Pad all other cycles to that many entries
numCycles = 0
count = 0
while count < 614: #find a way to automatically assign the limit later
    for fileName in os.listdir('Nasa Battery Data\Battery 5\Discharge Cycle\Headers'):

        #Increase counter until we've hit a cycle number that is in this folder
        #That means it is a discharge cycle
        cstring = str(count)
        #Create filePath variable to this file that we want to extract data from
        filePath = "B0005 cycle " + cstring+ " data with headers.csv"
        

        if fnmatch.fnmatch(fileName, filePath):
            numCycles +=1
    count +=1